<a href="https://colab.research.google.com/github/benmsanderson/prpattern/blob/main/CMIP6_get_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 4xCO2 read data


CMIP6

In [1]:
expts=['1pctCO2','abrupt-4xCO2','piControl','historical','ssp126','ssp585']

In [2]:
expts.index('piControl')

2

In [1]:
readdata=1
authdrive=1

In [23]:
datadir='/content/drive/MyDrive/colab_4xco2'

Desired interpolation grid

Desired experiments

In [3]:
flds=['tas']
expts=['1pctCO2','abrupt-4xCO2','piControl','historical','ssp126','ssp585']
calstrt=[True,True,True,False,False,False]
dbe=['CMIP','CMIP','CMIP','CMIP','ScenarioMIP','ScenarioMIP']

Install some stuff

In [5]:
!pip install --upgrade xarray zarr gcsfs cftime  nc-time-axis progress

  Using cached xarray-2022.3.0-py3-none-any.whl (870 kB)
  Using cached zarr-2.11.3-py3-none-any.whl (153 kB)
  Using cached gcsfs-2022.3.0-py2.py3-none-any.whl (25 kB)
  Using cached cftime-1.6.0-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (238 kB)
     |████████████████████████████████| 6.4 MB 5.2 MB/s eta 0:00:01
     |████████████████████████████████| 156 kB 65.7 MB/s eta 0:00:01
     |████████████████████████████████| 107 kB 60.6 MB/s eta 0:00:01
     |████████████████████████████████| 1.3 MB 68.0 MB/s eta 0:00:01
     |████████████████████████████████| 136 kB 72.4 MB/s eta 0:00:01
     |████████████████████████████████| 155 kB 72.8 MB/s eta 0:00:01
     |████████████████████████████████| 114 kB 44.5 MB/s eta 0:00:01
     |████████████████████████████████| 76 kB 5.5 MB/s  eta 0:00:01
     |████████████████████████████████| 1.0 MB 68.8 MB/s eta 0:00:01
     |████████████████████████████████| 308 kB 44.2 MB/s eta 0:00:01
     |████████████████████████████████| 121 kB 49

Import stuff

In [10]:
from matplotlib import pyplot as plt
import numpy as np
import numpy.matlib
import pandas as pd
import xarray as xr
import zarr
import gcsfs
import pickle
import cftime
from sys import getsizeof
from IPython.display import HTML, display
import time

#code for pretty progress bars
def progress(value, max=100):
    return HTML("""
        <progress
            value='{value}'
            max='{max}',
            style='width: 100%'
        >
            {value}
        </progress>
    """.format(value=value, max=max))


In [11]:
lon_out=np.arange(1,359,2)
lat_out=np.arange(-89,89,2)
lons_sub, lats_sub = np.meshgrid(lon_out,lat_out)

## Browse Catalog

The data catatalog is stored as a CSV file. Here we read it with Pandas.

In [12]:
df = pd.read_csv('https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv', low_memory=False)

Variables and experiments in database

In [13]:
vars=df.variable_id.unique()
vars.sort()
expts_full=df.experiment_id.unique()
expts_full.sort()

flds_full=df.variable_id.unique()
flds_full.sort()

In [14]:
df[(df["experiment_id"] == 'ssp585-bgc') & (df["variable_id"]=='fgco2')]

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year,version
241987,C4MIP,CNRM-CERFACS,CNRM-ESM2-1,ssp585-bgc,r1i1p1f2,Omon,fgco2,gn,gs://cmip6/CMIP6/C4MIP/CNRM-CERFACS/CNRM-ESM2-...,NaN,20190711
409547,C4MIP,MIROC,MIROC-ES2L,ssp585-bgc,r1i1p1f2,Omon,fgco2,gn,gs://cmip6/CMIP6/C4MIP/MIROC/MIROC-ES2L/ssp585...,NaN,20200124
464151,C4MIP,MIROC,MIROC-ES2L,ssp585-bgc,r1i1p1f2,Omon,fgco2,gr1,gs://cmip6/CMIP6/C4MIP/MIROC/MIROC-ES2L/ssp585...,NaN,20200731
497096,C4MIP,CSIRO,ACCESS-ESM1-5,ssp585-bgc,r1i1p1f1,Omon,fgco2,gn,gs://cmip6/CMIP6/C4MIP/CSIRO/ACCESS-ESM1-5/ssp...,NaN,20201016


Make dataframe for each experiment type and each field

In [15]:
df_all1=[]
for i, row in enumerate(expts):
  df_ta1=[]
  for j,fld in enumerate(flds):
    tmp = df.query("activity_id=='"+dbe[i]+"' & table_id == 'Amon' & variable_id == '"+fld+"' & experiment_id == '"+expts[i]+"'")
    df_ta1.append(tmp)
  df_all1.append(df_ta1)

Isolate unique models which have completed 4xco2

In [16]:
mdls1=df_all1[0][0].source_id.unique()
mdls1.sort()
mdls1

array(['ACCESS-CM2', 'ACCESS-ESM1-5', 'AWI-CM-1-1-MR', 'AWI-ESM-1-1-LR',
       'BCC-CSM2-MR', 'BCC-ESM1', 'CAMS-CSM1-0', 'CAS-ESM2-0', 'CESM2',
       'CESM2-FV2', 'CESM2-WACCM', 'CESM2-WACCM-FV2', 'CIESM',
       'CMCC-CM2-HR4', 'CMCC-CM2-SR5', 'CMCC-ESM2', 'CNRM-CM6-1',
       'CNRM-CM6-1-HR', 'CNRM-ESM2-1', 'CanESM5', 'CanESM5-CanOE',
       'E3SM-1-0', 'EC-Earth3', 'EC-Earth3-AerChem', 'EC-Earth3-CC',
       'EC-Earth3-Veg', 'FGOALS-f3-L', 'FGOALS-g3', 'FIO-ESM-2-0',
       'GFDL-CM4', 'GFDL-ESM4', 'GISS-E2-1-G', 'GISS-E2-1-H',
       'GISS-E2-2-G', 'GISS-E2-2-H', 'HadGEM3-GC31-LL', 'HadGEM3-GC31-MM',
       'ICON-ESM-LR', 'IITM-ESM', 'INM-CM4-8', 'INM-CM5-0',
       'IPSL-CM5A2-INCA', 'IPSL-CM6A-LR', 'KACE-1-0-G', 'KIOST-ESM',
       'MCM-UA-1-0', 'MIROC-ES2L', 'MIROC6', 'MPI-ESM-1-2-HAM',
       'MPI-ESM1-2-HR', 'MPI-ESM1-2-LR', 'MRI-ESM2-0', 'NESM3', 'NorCPM1',
       'NorESM2-LM', 'NorESM2-MM', 'SAM0-UNICON', 'TaiESM1',
       'UKESM1-0-LL'], dtype=object)

Make some empty dataframes to store concise list

In [17]:
cnames=df_all1[0][0].columns
df_all=[]
for i, exp in enumerate(expts):
  tmp=[]
  for j,fld  in enumerate(flds):
    tmp.append(pd.DataFrame(columns=cnames))
  df_all.append(tmp)

Now get 1 ensemble member for each model, if it exists, for each experiment.  Only add to dataframe df_ta if we have a full set of experiments

In [18]:
mdls=[]
n=0
for j, mdl in enumerate(mdls1):
    tmpdf=[]
    nruns=[]
    for i, ext in enumerate(expts):
        #find first variable for expt/model
        for j, fld in enumerate(flds):
          tmp=df_all1[i][j].query("source_id=='"+mdl+"'")
          nruns.append(tmp.shape[0])
    #is there at least 1 run per experiment,with all fields?
    if min(nruns)>=1:
      #point to the entry for 1st run, first variable for each expt
      for i, ext in enumerate(expts):
        mmb=df_all1[i][0]['member_id'].values[0]
        for j, fld in enumerate(flds):
          tt = df_all1[i][j].query("source_id=='"+mdl+"' & table_id == 'Amon'")
          df_all[i][j].loc[n]=tt.values[0]
      #add model to final list
      mdls.append(mdl)
      n=n+1
    
    

In [24]:
pickle.dump(mdls, open( datadir+"/mdls.pkl", "wb" ) )


In [25]:
df_all[1][2].loc(1)

## Load Data

Load Google file system


In [26]:
# load Google cloud storage
gcs = gcsfs.GCSFileSystem(token='anon')

Loop through zstore links, use zarr to open


In [27]:
nm=len(mdls)
nf=len(flds)
ne=len(expts)

In [ ]:

if readdata:
  out = display(progress(0, 1), display_id=True)
  dsall=[]
  for i,df_ta in enumerate(df_all):

    dsm=[]
    for j,df in enumerate(df_ta):
      ds=[]
      print(expts[i]+','+flds[j])
      for index, item in enumerate(df.zstore.values, start=0):
        mapper=gcs.get_mapper(item)
        ds.append(xr.open_zarr(mapper, decode_times=False))
        out.update(progress(index+j*nm+i*nm*nf, ne*nm*nf))
      dsm.append(ds)
    dsall.append(dsm)  

1pctCO2,tas
1pctCO2,rsut
1pctCO2,rlut
1pctCO2,rsdt
abrupt-4xCO2,tas
abrupt-4xCO2,rsut
abrupt-4xCO2,rlut
abrupt-4xCO2,rsdt
piControl,tas


concatenated dataarrays for ts, global mean


In [ ]:
 
if readdata:
  dall=[]
  out = display(progress(0, 1), display_id=True)
  for i,ds in enumerate(dsall,start=0):
    dexp=[]
    for j,dm in enumerate(ds):
      print(expts[i]+','+flds[j])
      for index, dd in enumerate(dm, start=0):
          if 'longitude' in dd.keys():
            dd=dd.rename({'longitude': 'lon','latitude': 'lat'})
          if 'latitude' in dd.coords:
            dd=dd.drop('latitude')  
            dd=dd.drop_dims('latitude')
       
          tmp=dd[flds[j]][:4800,:,:].interp(lon=lon_out,lat=lat_out, kwargs={"fill_value": "extrapolate"})
          if calstrt[i]:
            tmp.coords['time']=pd.date_range('1850-01-01', periods=tmp['time'].values.shape[0],freq='M')
          if tmp['time'].dtype=='float64' or  tmp['time'].dtype=='int64':
            tmp.coords['time']=num2date(tmp['time'].values,tmp['time'].units)     
          srm=tmp.groupby('time.year').mean('time')
          if index==0:
            dac=srm
          else:
            dac=xr.concat([dac,srm],'ens',coords='minimal',compat='override')
          out.update(progress(index+j*nm+i*nm*nf, ne*nm*nf))
      dexp.append(dac)
    dall.append(dexp)

1pctCO2,tas
1pctCO2,rsut
1pctCO2,rlut
1pctCO2,rsdt
abrupt-4xCO2,tas
abrupt-4xCO2,rsut
abrupt-4xCO2,rlut
abrupt-4xCO2,rsdt
piControl,tas
piControl,rsut
piControl,rlut
piControl,rsdt
historical,tas


/usr/local/lib/python3.7/dist-packages/dask/array/core.py:3908: PerformanceWarning: Increasing number of chunks by factor of 14
  **blockwise_kwargs,


historical,rsut


/usr/local/lib/python3.7/dist-packages/dask/array/core.py:3908: PerformanceWarning: Increasing number of chunks by factor of 14
  **blockwise_kwargs,


historical,rlut


...

ssp126,rsut
ssp126,rlut
ssp126,rsdt
ssp585,tas


/usr/local/lib/python3.7/dist-packages/dask/array/core.py:3908: PerformanceWarning: Increasing number of chunks by factor of 11
  **blockwise_kwargs,
/usr/local/lib/python3.7/dist-packages/dask/array/core.py:3908: PerformanceWarning: Increasing number of chunks by factor of 17
  **blockwise_kwargs,


ssp585,rsut
ssp585,rlut


/usr/local/lib/python3.7/dist-packages/dask/array/core.py:3908: PerformanceWarning: Increasing number of chunks by factor of 29
  **blockwise_kwargs,


ssp585,rsdt


Find complete runs




In [ ]:

if readdata:
  out = display(progress(0, 1), display_id=True)
  for i,d in enumerate(dall,start=0):
    tmp=xr.merge(d[:])
    tmp.to_netcdf(datadir+'/colab_4xco2/'+expts[i]+'.nc')
    #lat=tmp.tas.lat
    #weights = np.cos(np.deg2rad(lat))
    #weights.name = "weights"
    #tmp_gm=tmp.weighted(weights).mean(dim='lat').mean(dim='lon')
    #tmp_gm.to_netcdf('/content/drive/MyDrive/colab_4xco2/'+expts[i]+'_gm.nc')
    out.update(progress(i,ne))



/usr/local/lib/python3.7/dist-packages/dask/array/core.py:3908: PerformanceWarning: Increasing number of chunks by factor of 34
  **blockwise_kwargs,
/usr/local/lib/python3.7/dist-packages/dask/array/core.py:3908: PerformanceWarning: Increasing number of chunks by factor of 34
  **blockwise_kwargs,
/usr/local/lib/python3.7/dist-packages/dask/array/core.py:3908: PerformanceWarning: Increasing number of chunks by factor of 34
  **blockwise_kwargs,
/usr/local/lib/python3.7/dist-packages/dask/array/core.py:3908: PerformanceWarning: Increasing number of chunks by factor of 34
  **blockwise_kwargs,
/usr/local/lib/python3.7/dist-packages/dask/array/core.py:3908: PerformanceWarning: Increasing number of chunks by factor of 34
  **blockwise_kwargs,
/usr/local/lib/python3.7/dist-packages/dask/array/core.py:3908: PerformanceWarning: Increasing number of chunks by factor of 34
  **blockwise_kwargs,
/usr/local/lib/python3.7/dist-packages/dask/array/core.py:3908: PerformanceWarning: Increasing numbe

In [ ]:

  
if readdata:
  out = display(progress(0, 1), display_id=True)
  for i,d in enumerate(dall,start=0):
    tmp=xr.open_dataset(datadir+'/'+expts[i]+'.nc')
    lat=tmp.tas.lat
    weights = np.cos(np.deg2rad(lat))
    weights.name = "weights"
    tmp_gm=tmp.weighted(weights).mean(dim='lat').mean(dim='lon')
    tmp_gm.to_netcdf(datadir+'/'+expts[i]+'_gm.nc')
    out.update(progress(i,ne))
